# Create a Dataset using Databricks

Steps
* databricks driver is connected to our postgres DB
* databricks driver reads the scan artifacts from postgres DB
* databricks driver copies all scan artifact blobs to DBFS
* databricks worker can open and process blobs

In [0]:
! pip install scikit-image
! pip install tqdm
! pip install azure-storage-blob

Requirement already satisfied: scikit-image in /databricks/python3/lib/python3.7/site-packages (0.18.1)
Requirement already satisfied: pillow!=7.1.0,!=7.1.1,>=4.3.0 in /databricks/python3/lib/python3.7/site-packages (from scikit-image) (8.2.0)
Requirement already satisfied: networkx>=2.0 in /databricks/python3/lib/python3.7/site-packages (from scikit-image) (2.5.1)
Requirement already satisfied: matplotlib!=3.0.0,>=2.0.0 in /databricks/python3/lib/python3.7/site-packages (from scikit-image) (3.1.3)
Requirement already satisfied: scipy>=1.0.1 in /databricks/python3/lib/python3.7/site-packages (from scikit-image) (1.4.1)
Requirement already satisfied: PyWavelets>=1.1.1 in /databricks/python3/lib/python3.7/site-packages (from scikit-image) (1.1.1)
Requirement already satisfied: imageio>=2.3.0 in /databricks/python3/lib/python3.7/site-packages (from scikit-image) (2.9.0)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.7/site-packages (from scikit-image) (1.18.1)
Requirement already satisfied: tifffile>=2019.7.26 in /databricks/python3/lib/python3.7/site-packages (from scikit-image) (2021.4.8)
Requirement already satisfied: decorator<5,>=4.3 in /databricks/python3/lib/python3.7/site-packages (from networkx>=2.0->scikit-image) (4.4.1)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.7/site-packages (from matplotlib!=3.0.0,>=2.0.0->scikit-image) (0.10.0)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib!=3.0.0,>=2.0.0->scikit-image) (2.4.6)
Requirement already satisfied: python-dateutil>=2.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib!=3.0.0,>=2.0.0->scikit-image) (2.8.1)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib!=3.0.0,>=2.0.0->scikit-image) (1.1.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from cycler>=0.10->matplotlib!=3.0.0,>=2.0.0->scikit-image) (1.14.0)
Requirement already satisfied: setuptools in /usr/local/lib/python3.7/dist-packages (from kiwisolver>=1.0.1->matplotlib!=3.0.0,>=2.0.0->scikit-image) (45.2.0)
 WARNING: You are using pip version 20.0.2; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command. 
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.7/site-packages (4.60.0)
 WARNING: You are using pip version 20.0.2; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command. 
Requirement already satisfied: azure-storage-blob in /databricks/python3/lib/python3.7/site-packages (12.8.1)
Requirement already satisfied: msrest>=0.6.18 in /databricks/python3/lib/python3.7/site-packages (from azure-storage-blob) (0.6.21)
Requirement already satisfied: cryptography>=2.1.4 in /databricks/python3/lib/python3.7/site-packages (from azure-storage-blob) (2.8)
Requirement already satisfied: azure-core<2.0.0,>=1.10.0 in /databricks/python3/lib/python3.7/site-packages (from azure-storage-blob) (1.13.0)
Requirement already satisfied: isodate>=0.6.0 in /databricks/python3/lib/python3.7/site-packages (from msrest>=0.6.18->azure-storage-blob) (0.6.0)
Requirement already satisfied: requests~=2.16 in /databricks/python3/lib/python3.7/site-packages (from msrest>=0.6.18->azure-storage-blob) (2.22.0)
Requirement already satisfied: requests-oauthlib>=0.5.0 in /databricks/python3/lib/python3.7/site-packages (from msrest>=0.6.18->azure-storage-blob) (1.3.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from msrest>=0.6.18->azure-storage-blob) (2020.12.5)
Requirement already satisfied: cffi!=1.11.3,>=1.8 in /databricks/python3/lib/python3.7/site-packages (from cryptography>=

In [0]:
from datetime import datetime, timezone
import os
from pathlib import Path
import pickle
from typing import Tuple, List
import zipfile

import numpy as np
import pandas as pd
import psycopg2
from skimage.transform import resize
from tqdm import tqdm
from azure.storage.blob import BlobServiceClient

In [0]:
# Constants
ENV_PROD = "env_prod"
ENV_SANDBOX = "env_sandbox"

In [0]:
# Configuration
DEBUG = False
ENV = ENV_SANDBOX

MOUNT_POINT = f"/mnt/{ENV}_input"
MOUNT_DATASET = f"/mnt/{ENV}_dataset"
DBFS_DIR = f"/tmp/{ENV}"

## Access SQL database to find all the scans/artifacts of interest

#### SQL query

We build our SQL query, so that we get all the required information for ML:
- the artifacts (depthmap, RGB, pointcloud)
- the targets (measured height, weight, and MUAC)

The ETL packet shows which tables are involved 

![image info](https://dev.azure.com/cgmorg/e5b67bad-b36b-4475-bdd7-0cf6875414df/_apis/git/repositories/465970a9-a8a5-4223-81c1-2d3f3bd4ab26/Items?path=%2F.attachments%2Fcgm-solution-architecture-etl-draft-ETL-samplling-71a42e64-72c4-4360-a741-1cfa24622dce.png&download=false&resolveLfs=true&%24format=octetStream&api-version=5.0-preview.1&sanitize=true&versionDescriptor.version=wikiMaster)

The query will produce one artifact per row.

In [0]:
SECRET_SCOPE = "cgm-ml-ci-dev-databricks-secret-scope"
if ENV == ENV_SANDBOX: 
    host = "cgm-ml-ci-dev-mlapi-psql.postgres.database.azure.com"
    user = dbutils.secrets.get(scope=SECRET_SCOPE, key="psql-username")
    password = dbutils.secrets.get(scope=SECRET_SCOPE, key="psql-password")
elif ENV == ENV_PROD: 
    host = "cgm-ml-ci-prod-mlapi-psql.postgres.database.azure.com"
    user = dbutils.secrets.get(scope=SECRET_SCOPE, key="prod-psql-username")
    password = dbutils.secrets.get(scope=SECRET_SCOPE, key="prod-psql-password")
else: 
    raise Exception(f"Unknown environment: {ENV}")

conn = psycopg2.connect(host=host, database='cgm-ml', user=user, password=password)

In [0]:
cur = conn.cursor()

In [0]:
SQL_QUERY = """
SELECT f.file_path, f.created as timestamp, 
       s.id as scan_id, s.scan_type_id as scan_step, 
       m.height, m.weight, m.muac,
       a.ord as order_number
FROM file f 
INNER JOIN artifact a ON f.id = a.file_id 
INNER JOIN scan s     ON s.id = a.scan_id 
INNER JOIN measure m  ON m.person_id = s.person_id
WHERE a.format = 'depth'
"""
cur.execute(SQL_QUERY)

# Get a query_result row
if DEBUG:
    query_result_one: Tuple[str] = cur.fetchone()
    file_path = query_result_one[0]; file_path
  
# Get multiple query_result rows
NUM_ARTIFACTS = 3000  # None
query_results: List[Tuple[str]] = cur.fetchall() if NUM_ARTIFACTS is None else cur.fetchmany(NUM_ARTIFACTS)

**Explanation of a file_path**

The SQL result provides file_paths which have this format

```
Example: '1618896404960/2fe0ee0e-daf0-45a4-931e-cfc7682e1ce6'
Format: f'{unix-timeatamp}/{random uuid}'
```

In [0]:
df = pd.DataFrame(query_results, columns=list(map(lambda x: x.name, cur.description)))
print(df.shape)
df.head()

(3000, 8)
Out[49]:

,file_path,timestamp,scan_id,scan_step,height,weight,muac,order_number
0,1619098473303/ddf9c30e-12e1-4886-8440-468106f6...,2021-04-22 13:34:33.302557,c571de02-a723-11eb-8845-bb6589a1fbe8,102,50.0,20.555,10.0,3
1,1619098473303/ddf9c30e-12e1-4886-8440-468106f6...,2021-04-22 13:34:33.302557,bb9c2ac0-a723-11eb-8845-db2956a520c8,102,50.0,20.555,10.0,3
2,1619098473303/ddf9c30e-12e1-4886-8440-468106f6...,2021-04-22 13:34:33.302557,68ee33a0-a722-11eb-a9c9-e789bc71bdb8,102,50.0,20.555,10.0,3
3,1619103910489/d95dec40-ce59-4ae7-884c-e4d88fe6...,2021-04-22 15:05:10.592732,dbf2e422-a37d-11eb-a0ce-8f4b16bac212,101,72.0,12.000,12.0,0
4,1619103910489/d95dec40-ce59-4ae7-884c-e4d88fe6...,2021-04-22 15:05:10.592732,7d508300-a37d-11eb-9eb2-2b29b077e241,101,72.0,12.000,12.0,0


In [0]:
col2idx = {col.name: i for i, col in enumerate(cur.description)}; print(col2idx)
idx2col = {i: col.name for i, col in enumerate(cur.description)}; print(idx2col)

{'file_path': 0, 'timestamp': 1, 'scan_id': 2, 'scan_step': 3, 'height': 4, 'weight': 5, 'muac': 6, 'order_number': 7}
{0: 'file_path', 1: 'timestamp', 2: 'scan_id', 3: 'scan_step', 4: 'height', 5: 'weight', 6: 'muac', 7: 'order_number'}

# Copy mounted files to DBFS

In order for databricks to process the blob data, we need to transfer it to the DBFS of the databricks cluster.

Note:
* It is not be enough to mount the blob storage. 
* Copying from mount is very very slow.


## Download blobs 

We use [Manage blobs Python SDK](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python#download-blobs)
to download blobs directly from the Storage Account(SA) to [DBFS](https://docs.databricks.com/data/databricks-file-system.html).

In [0]:
if ENV == ENV_SANDBOX: 
    CONNECTION_STR = dbutils.secrets.get(scope=SECRET_SCOPE, key="dev-sa-connection-string")
    STORAGE_ACCOUNT_NAME = "cgmmlcidevmlapisa"
    CONTAINER_NAME = "cgm-result"
elif ENV == ENV_PROD: 
    raise Exception("Not yet setup this SA connection string")
else: 
    raise Exception(f"Unknown environment: {ENV}")

In [0]:
BLOB_SERVICE_CLIENT = BlobServiceClient.from_connection_string(CONNECTION_STR)

In [0]:
if DEBUG:
    file_name = query_results[0][0]; print(file_name)
    with open('asjj.txt', "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

In [0]:
def download_from_blob_storage(src, dest, container):
    blob_client = BLOB_SERVICE_CLIENT.get_blob_client(container=container, blob=file_path)
    Path(dest).parent.mkdir(parents=True, exist_ok=True)
    with open(dest, "wb") as download_file:
        content = blob_client.download_blob().readall()
        download_file.write(content)

In [0]:
file_path_idx = col2idx['file_path']
for res in tqdm(query_results):
    file_path = res[file_path_idx]
    download_from_blob_storage(src=file_path, dest=f"dbfs:{DBFS_DIR}/{file_path}", container=CONTAINER_NAME)

0%| | 0/3000 [00:00<?, ?it/s] 0%| | 1/3000 [00:00<05:45, 8.67it/s] 0%| | 13/3000 [00:00<00:44, 67.60it/s] 1%| | 27/3000 [00:00<00:30, 97.23it/s] 1%| | 37/3000 [00:00<00:30, 97.57it/s] 2%|▏ | 52/3000 [00:00<00:25, 113.48it/s] 2%|▏ | 67/3000 [00:00<00:24, 122.16it/s] 3%|▎ | 80/3000 [00:00<00:23, 123.00it/s] 3%|▎ | 93/3000 [00:00<00:24, 119.28it/s] 4%|▎ | 105/3000 [00:01<00:27, 105.81it/s] 4%|▍ | 119/3000 [00:01<00:25, 114.32it/s] 4%|▍ | 131/3000 [00:01<00:25, 113.70it/s] 5%|▍ | 143/3000 [00:01<00:26, 108.74it/s] 5%|▌ | 155/3000 [00:01<00:35, 79.11it/s]  6%|▌ | 165/3000 [00:01<00:34, 81.41it/s] 6%|▌ | 176/3000 [00:01<00:32, 86.81it/s] 6%|▌ | 186/3000 [00:01<00:31, 89.76it/s] 7%|▋ | 196/3000 [00:02<00:30, 91.34it/s] 7%|▋ | 208/3000 [00:02<00:28, 97.70it/s] 7%|▋ | 219/3000 [00:02<00:27, 100.77it/s] 8%|▊ | 231/3000 [00:02<00:26, 105.88it/s] 8%|▊ | 246/3000 [00:02<00:23, 116.73it/s] 9%|▊ | 261/3000 [00:02<00:21, 125.02it/s] 9%|▉ | 274/3000 [00:02<00:22, 119.63it/s] 10%|▉ | 287/3000 [00:02<00:22, 121.99it/s] 10%|█ | 300/3000 [00:02<00:21, 124.21it/s] 10%|█ | 313/3000 [00:02<00:21, 122.65it/s] 11%|█ | 326/3000 [00:03<00:23, 115.33it/s] 11%|█▏ | 338/3000 [00:03<00:24, 107.43it/s] 12%|█▏ | 349/3000 [00:03<00:24, 107.98it/s] 12%|█▏ | 360/3000 [00:03<00:25, 102.38it/s] 12%|█▏ | 372/3000 [00:03<00:24, 106.08it/s] 13%|█▎ | 385/3000 [00:03<00:23, 110.92it/s] 13%|█▎ | 397/3000 [00:03<00:23, 111.60it/s] 14%|█▎ | 411/3000 [00:03<00:21, 118.11it/s] 14%|█▍ | 424/3000 [00:03<00:22, 116.43it/s] 15%|█▍ | 436/3000 [00:04<00:22, 113.37it/s] 15%|█▍ | 448/3000 [00:04<00:22, 113.65it/s] 15%|█▌ | 461/3000 [00:04<00:21, 117.10it/s] 16%|█▌ | 473/3000 [00:04<00:22, 112.09it/s] 16%|█▌ | 487/3000 [00:04<00:21, 119.04it/s] 17%|█▋ | 501/3000 [00:04<00:20, 123.22it/s] 17%|█▋ | 514/3000 [00:04<00:20, 122.60it/s] 18%|█▊ | 528/3000 [00:04<00:19, 126.21it/s] 18%|█▊ | 541/3000 [00:05<00:39, 62.07it/s]  18%|█▊ | 552/3000 [00:05<00:35, 69.39it/s] 19%|█▉ | 565/3000 [00:05<00:30, 80.14it/s] 19%|█▉ | 578/3000 [00:05<00:26, 90.63it/s] 20%|█▉ | 590/3000 [00:05<00:26, 89.49it/s] 20%|██ | 601/3000 [00:05<00:26, 90.36it/s] 20%|██ | 614/3000 [00:05<00:24, 99.10it/s] 21%|██ | 625/3000 [00:06<00:25, 92.60it/s] 21%|██ | 635/3000 [00:06<00:25, 93.94it/s] 22%|██▏ | 647/3000 [00:06<00:23, 99.64it/s] 22%|██▏ | 660/3000 [00:06<00:21, 106.38it/s] 22%|██▏ | 674/3000 [00:06<00:20, 113.48it/s] 23%|██▎ | 686/3000 [00:06<00:20, 113.60it/s] 23%|██▎ | 698/3000 [00:06<00:22, 103.32it/s] 24%|██▎ | 709/3000 [00:06<00:24, 93.24it/s]  24%|██▍ | 720/3000 [00:07<00:23, 96.80it/s] 24%|██▍ | 734/3000 [00:07<00:21, 106.51it/s] 25%|██▍ | 748/3000 [00:07<00:19, 114.08it/s] 25%|██▌ | 763/3000 [00:07<00:18, 123.96it/s] 26%|██▌ | 777/3000 [00:07<00:17, 123.95it/s] 26%|██▋ | 790/3000 [00:07<00:17, 123.87it/s] 27%|██▋ | 804/3000 [00:07<00:17, 127.51it/s] 27%|██▋ | 818/3000 [00:07<00:16, 130.91it/s] 28%|██▊ | 833/3000 [00:07<00:16, 134.58it/s] 28%|██▊ | 847/3000 [00:07<00:15, 134.87it/s] 29%|██▊ | 861/3000 [00:08<00:15, 134.19it/s] 29%|██▉ | 875/3000 [00:08<00:15, 135.62it/s] 30%|██▉ | 890/3000 [00:08<00:15, 137.51it/s] 30%|███ | 905/3000 [00:08<00:15, 138.88it/s] 31%|███ | 919/3000 [00:08<00:15, 138.58it/s] 31%|███ | 933/3000 [00:08<00:15, 137.29it/s] 32%|███▏ | 948/3000 [00:08<00:15, 136.47it/s] 32%|███▏ | 964/3000 [00:08<00:14, 140.83it/s] 33%|███▎ | 980/3000 [00:08<00:13, 144.83it/s] 33%|███▎ | 996/3000 [00:09<00:13, 147.10it/s] 34%|███▎ | 1011/3000 [00:09<00:13, 145.02it/s] 34%|███▍ | 1027/3000 [00:09<00:13, 148.21it/s] 35%|███▍ | 1043/3000 [00:09<00:13, 150.32it/s] 35%|███▌ | 1059/3000 [00:09<00:13, 148.03it/s] 36%|███▌ | 1074/3000 [00:09<00:13, 146.66it/s] 36%|███▋ | 1089/3000 [00:09<00:13, 144.42it/s] 37%|███▋ | 1104/3000 [00:09<00:13, 139.70it/s] 37%|███▋ | 1119/3000 [00:09<00:13, 135.02it/s] 38%|███▊ | 1133/3000 [00:10<00:14, 130.82it/s] 38%|███▊ | 1147/3000 [00:10<00:13, 132.46it/s] 39%|███▊ | 1161/3000 [00:10<00:1

# Transform ZIP into pickle

Here we document the format of the artifact path

```
f"scans/1583462505-43bak4gvfa/101/pc_1583462505-43bak4gvfa_1591122173510_101_002.p"
f"qrcode/{scan_id}/{scan_step}/pc_{scan_id}_{timestamp}_{scan_step}_{order_number}.p"
```

Idea for a future format could be to include person_id like so:
```
f"qrcode/{person_id}/{scan_step}/pc_{scan_id}_{timestamp}_{scan_step}_{order_number}.p"
```

## Preprocessing utilities

In order to preprocess ZIP file to extract a depthmap, we use this code:
[preprocessing.py](https://github.com/Welthungerhilfe/cgm-rg/blob/92efa0febb91c9656ce8e5dbfad953ff7ce721a9/src/utils/preprocessing.py#L12)

[file of minor importance](https://github.com/Welthungerhilfe/cgm-ml/blob/c8be9138e025845bedbe7cfc0d131ef668e01d4b/old/cgm_database/command_preprocess.py#L92)

In [0]:
WIDTH = 240
HEIGHT = 180
NORMALIZATION_VALUE = 7.5
IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH = HEIGHT, WIDTH

def load_depth(fpath: str) -> Tuple[bytes, int, int, float, float]:
    """Take ZIP file and extract depth and metadata
    Args:
        fpath (str): File path to the ZIP
    Returns:
        depth_data (bytes): depthmap data
        width(int): depthmap width in pixel
        height(int): depthmap height in pixel
        depth_scale(float)
        max_confidence(float)
    """

    with zipfile.ZipFile(fpath) as z:
        with z.open('data') as f:
            # Example for a first_line: '180x135_0.001_7_0.57045287_-0.0057296_0.0022602521_0.82130724_-0.059177425_0.0024800065_0.030834956'
            first_line = f.readline().decode().strip()

            file_header = first_line.split("_")

            # header[0] example: 180x135
            width, height = file_header[0].split("x")
            width, height = int(width), int(height)
            depth_scale = float(file_header[1])
            max_confidence = float(file_header[2])

            depth_data = f.read()
    return depth_data, width, height, depth_scale, max_confidence


def parse_depth(tx: int, ty: int, data: bytes, depth_scale: float, width: int) -> float:
    assert isinstance(tx, int)
    assert isinstance(ty, int)

    depth = data[(ty * width + tx) * 3 + 0] << 8
    depth += data[(ty * width + tx) * 3 + 1]

    depth *= depth_scale
    return depth

def preprocess_depthmap(depthmap):
    return depthmap.astype("float32")

def preprocess(depthmap):
    depthmap = preprocess_depthmap(depthmap)
    depthmap = depthmap / NORMALIZATION_VALUE
    depthmap = resize(depthmap, (IMAGE_TARGET_HEIGHT, IMAGE_TARGET_WIDTH))
    depthmap = depthmap.reshape((depthmap.shape[0], depthmap.shape[1], 1))
    return depthmap
  
def prepare_depthmap(data: bytes, width: int, height: int, depth_scale: float) -> np.array:
    """Convert bytes array into np.array"""
    output = np.zeros((width, height, 1))
    for cx in range(width):
        for cy in range(height):
            # depth data scaled to be visible
            output[cx][height - cy - 1] = parse_depth(cx, cy, data, depth_scale, width)
    arr = np.array(output, dtype='float32')
    return arr.reshape(width, height)

def get_depthmaps(fpaths):
    depthmaps = []
    for fpath in fpaths:
        data, width, height, depthScale, _ = load_depth(fpath)
        depthmap = prepare_depthmap(data, width, height, depthScale)
        depthmap = preprocess(depthmap)
        depthmaps.append(depthmap)

    depthmaps = np.array(depthmaps)
    return depthmaps

## Transform utilities

In [0]:
def create_and_save_pickle(zip_input_full_path, timestamp, scan_id, scan_step, target_tuple, order_number):
    """Side effect: Saves and returns file path"""
    depthmaps = get_depthmaps([zip_input_full_path])
    if DEBUG:
        print(depthmaps.shape, depthmaps[0,0,0,0])
    
    pickle_output_path = f"qrcode/{scan_id}/{scan_step}/pc_{scan_id}_{timestamp}_{scan_step}_{order_number}.p"  # '/tmp/abc.p'
    pickle_output_full_path = f"/dbfs{DBFS_DIR}/{pickle_output_path}"
    Path(pickle_output_full_path).parent.mkdir(parents=True, exist_ok=True)
    pickle.dump((depthmaps, np.array(target_tuple)), open(pickle_output_full_path, "wb"))
    return pickle_output_full_path

In [0]:
def process_artifact_tuple(artifact_tuple):
    """Side effect: Saves and returns file path"""
    artifact_dict = {idx2col[i]: el for i, el in enumerate(artifact_tuple)}
    if DEBUG:
        print('artifact_dict', artifact_dict)
    target_tuple = (artifact_dict['height'], artifact_dict['weight'], artifact_dict['muac'])
    zip_input_full_path = f"/dbfs{DBFS_DIR}/{artifact_dict['file_path']}"

    pickle_output_full_path = create_and_save_pickle(
        zip_input_full_path=zip_input_full_path,
        timestamp=artifact_dict['timestamp'],
        scan_id=artifact_dict['scan_id'],
        scan_step=artifact_dict['scan_step'],
        target_tuple=target_tuple,
        order_number=artifact_dict['order_number'],
    )
    return pickle_output_full_path

## Transform

In [0]:
if DEBUG:
  # Process on the spark driver
  for query_result in query_results[:2]:
      _ = process_artifact_tuple(query_result) 

In [0]:
rdd = spark.sparkContext.parallelize(query_results, 2)
if DEBUG:
    print(rdd.top(3))  # Inspect first items
    print(rdd.count())

In [0]:
rdd_processed = rdd.map(process_artifact_tuple)
processed_fnames = rdd_processed.collect()  # processed_fnames = rdd_processed.take(12)
print(processed_fnames[:3])

['/dbfs/tmp/env_sandbox/qrcode/c571de02-a723-11eb-8845-bb6589a1fbe8/102/pc_c571de02-a723-11eb-8845-bb6589a1fbe8_2021-04-22 13:34:33.302557_102_3.p', '/dbfs/tmp/env_sandbox/qrcode/bb9c2ac0-a723-11eb-8845-db2956a520c8/102/pc_bb9c2ac0-a723-11eb-8845-db2956a520c8_2021-04-22 13:34:33.302557_102_3.p', '/dbfs/tmp/env_sandbox/qrcode/68ee33a0-a722-11eb-a9c9-e789bc71bdb8/102/pc_68ee33a0-a722-11eb-a9c9-e789bc71bdb8_2021-04-22 13:34:33.302557_102_3.p']

# Upload to blob storage

We use [Manage blobs Python SDK](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python#upload-blobs-to-a-container)
to upload blobs

In [0]:
if ENV == ENV_SANDBOX: 
    STORAGE_ACCOUNT_NAME = "cgmmlcidevmlapisa"
    CONTAINER_NAME_DATASET = "cgm-datasets"
    CONNECT_STR_DATASET = CONNECTION_STR
elif ENV == ENV_PROD: 
    raise Exception("Not yet setup this SA connection string")
else: 
    raise Exception(f"Unknown environment: {ENV}")

In [0]:
def remove_prefix(text, prefix):
    if text.startswith(prefix): return text[len(prefix):]
    return text

prefix = f"/dbfs{DBFS_DIR}/"

def upload_to_blob_storage(src, dest, container, directory):
    blob_client = BLOB_SERVICE_CLIENT.get_blob_client(container=container, blob=os.path.join(directory,dest))
    # print(f"Uploading to Azure Storage as blob: {directory}/{dest}")
    with open(src, "rb") as data:
        blob_client.upload_blob(data, overwrite=False)

In [0]:
if DEBUG:
    print(len(processed_fnames))
    processed_fnames[0]
    full_name = processed_fnames[0]
    assert prefix in full_name
    fname = remove_prefix(full_name, prefix)
    upload_to_blob_storage(src=full_name, dest=fname, container=CONTAINER_NAME_DATASET)

In [0]:
DATASET_NAME = "dataset"
directory = datetime.now(timezone.utc).strftime(f"{DATASET_NAME}-%Y-%m-%d-%H-%M-%S")
for full_name in tqdm(processed_fnames):
    assert prefix in full_name
    fname = remove_prefix(full_name, prefix)
    upload_to_blob_storage(src=full_name, dest=fname, container=CONTAINER_NAME_DATASET, directory=directory)

0%| | 0/100 [00:00<?, ?it/s] 2%|▏ | 2/100 [00:00<00:06, 15.65it/s] 5%|▌ | 5/100 [00:00<00:04, 19.20it/s] 9%|▉ | 9/100 [00:00<00:03, 24.89it/s] 12%|█▏ | 12/100 [00:00<00:03, 23.21it/s] 16%|█▌ | 16/100 [00:00<00:03, 26.49it/s] 19%|█▉ | 19/100 [00:00<00:03, 26.98it/s] 22%|██▏ | 22/100 [00:00<00:03, 23.86it/s] 27%|██▋ | 27/100 [00:01<00:02, 27.75it/s] 31%|███ | 31/100 [00:01<00:02, 30.46it/s] 35%|███▌ | 35/100 [00:01<00:02, 29.06it/s] 39%|███▉ | 39/100 [00:01<00:01, 31.42it/s] 43%|████▎ | 43/100 [00:01<00:02, 28.44it/s] 46%|████▌ | 46/100 [00:01<00:01, 27.70it/s] 49%|████▉ | 49/100 [00:01<00:01, 26.57it/s] 52%|█████▏ | 52/100 [00:01<00:01, 27.40it/s] 56%|█████▌ | 56/100 [00:02<00:01, 28.20it/s] 60%|██████ | 60/100 [00:02<00:01, 28.68it/s] 63%|██████▎ | 63/100 [00:02<00:01, 28.86it/s] 66%|██████▌ | 66/100 [00:02<00:01, 27.85it/s] 70%|███████ | 70/100 [00:02<00:01, 28.96it/s] 73%|███████▎ | 73/100 [00:02<00:00, 29.16it/s] 77%|███████▋ | 77/100 [00:02<00:00, 31.24it/s] 81%|████████ | 81/100 [00:02<00:00, 31.18it/s] 85%|████████▌ | 85/100 [00:03<00:00, 28.97it/s] 88%|████████▊ | 88/100 [00:03<00:00, 26.91it/s] 91%|█████████ | 91/100 [00:03<00:00, 27.22it/s] 95%|█████████▌| 95/100 [00:03<00:00, 29.82it/s] 99%|█████████▉| 99/100 [00:03<00:00, 31.51it/s]100%|██████████| 100/100 [00:03<00:00, 28.21it/s]